In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import datetime

In [ ]:
try:
    orders = pd.read_csv('olist_orders_dataset.csv')
    items = pd.read_csv('olist_order_items_dataset.csv')
    customers = pd.read_csv('olist_customers_dataset.csv')

    print("✅ Success! All 3 files are loaded.")
    print(f"Orders count: {len(orders)}")
    print(f"Items count: {len(items)}")
    print(f"Customers count: {len(customers)}")

except FileNotFoundError:
    print("❌ Error: I can't find the files.")

✅ Success! All 3 files are loaded.
Orders count: 59058
Items count: 76182
Customers count: 99441


In [ ]:
# --- ACTION 1: TIME TRAVEL FIX (Convert to Datetime) ---
# We need this to calculate "Recency" later
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])

# --- ACTION 2: QUALITY CONTROL (Filter for Delivered) ---
orders_clean = orders[orders['order_status'] == 'delivered']

# --- ACTION 3: THE DOUBLE BRIDGE (Merging) ---
orders_items = pd.merge(orders_clean, items, on='order_id', how='inner')
master_df = pd.merge(orders_items, customers, on='customer_id', how='inner')

# --- VERIFICATION ---
print("✅ Phase 1 Complete!")
print(f"Master DataFrame Shape: {master_df.shape}")
print("\n--- Preview of the Master Data ---")
# We select just the 3 columns we care about to verify they exist
print(master_df[['customer_unique_id', 'order_purchase_timestamp', 'price']].head())

✅ Phase 1 Complete!
Master DataFrame Shape: (44192, 18)

--- Preview of the Master Data ---
                 customer_unique_id order_purchase_timestamp  price
0  af07308b275d755c9edb36a90c618231      2018-07-24 20:41:37  118.7
1  3a653a41f6f9fc3d2a113cf8398680e8      2018-08-08 08:38:49  159.9
2  7c142cf63193a1473d2e66489a9ae977      2017-11-18 19:28:06   45.0
3  72632f0f9dd73dfee390c9b22eb56dd6      2018-02-13 21:18:39   19.9
4  80bb27c7c16e8f973207a5086ab329e2      2017-07-09 21:57:05  147.9


In [ ]:
master_df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,1,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.7,22.76,af07308b275d755c9edb36a90c618231,47813,barreiras,BA
1,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,1,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-13 08:55:23,159.9,19.22,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO
2,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,1,d0b61bfb1de832b15ba9d266ca96e5b0,66922902710d126a0e7d26b0e3805106,2017-11-23 19:45:59,45.0,27.20,7c142cf63193a1473d2e66489a9ae977,59296,sao goncalo do amarante,RN
3,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,1,65266b2da20d04dbe00c5c2d3bb7859e,2c9e548be18521d1c43cde1c582c6de8,2018-02-19 20:31:37,19.9,8.72,72632f0f9dd73dfee390c9b22eb56dd6,9195,santo andre,SP
4,a4591c265e18cb1dcee52889e2d8acc3,503740e9ca751ccdda7ba28e9ab8f608,delivered,2017-07-09 21:57:05,2017-07-09 22:10:13,2017-07-11 14:58:04,2017-07-26 10:57:55,2017-08-01 00:00:00,1,060cb19345d90064d1015407193c233d,8581055ce74af1daba164fdbd55a40de,2017-07-13 22:10:13,147.9,27.36,80bb27c7c16e8f973207a5086ab329e2,86320,congonhinhas,PR


In [ ]:
import datetime as dt

# --- LANGKAH 1: BUAT TANGGAL PATOKAN ---
max_date = master_df['order_purchase_timestamp'].max() + dt.timedelta(days=1)
print(f"Tanggal Patokan Analisis: {max_date}")

# --- LANGKAH 2: MENGHITUNG RFM (GROUPBY) ---
rfm_df = master_df.groupby('customer_unique_id').agg({
    'order_purchase_timestamp': lambda x: (max_date - x.max()).days,
    'order_id': 'count',
    'price': 'sum'
}).reset_index()

# --- LANGKAH 3: RAPIKAN TABEL ---
rfm_df.columns = ['customer_id', 'Recency', 'Frequency', 'Monetary']

# --- LANGKAH 4: CEK HASIL ---
print("\n--- Contoh 5 Data Teratas ---")
print(rfm_df.head())

print("\n--- Statistik Data (Perhatikan Kolom Frequency!) ---")
print(rfm_df.describe())

Tanggal Patokan Analisis: 2018-08-30 14:18:28

--- Contoh 5 Data Teratas ---
                        customer_id  Recency  Frequency  Monetary
0  00053a61a98854899e70ed204dd4bafe      183          2    382.00
1  0006fdc98a402fceb4eb0ee528f6a8d4      408          1     13.90
2  00082cbe03e478190aadbea78542e933      283          1     79.00
3  00090324bbad0e9342388303bb71ba0a      158          1     49.95
4  000d460961d6dbfa3ec6c9f5805769e1      234          1     28.90

--- Statistik Data (Perhatikan Kolom Frequency!) ---
            Recency     Frequency      Monetary
count  38178.000000  38178.000000  38178.000000
mean     238.076405      1.157525    139.093348
std      151.875123      0.570297    219.169810
min        1.000000      1.000000      0.000000
25%      115.000000      1.000000     47.000000
50%      219.000000      1.000000     89.000000
75%      346.000000      1.000000    150.000000
max      695.000000     18.000000  13440.000000


In [ ]:
# --- FASE 3: SCORING (PEMBERIAN NILAI 1-5) ---
r_labels = range(5, 0, -1) # [5, 4, 3, 2, 1]
f_labels = range(1, 6)     # [1, 2, 3, 4, 5]
m_labels = range(1, 6)     # [1, 2, 3, 4, 5]

# 2. Hitung R Score (Recency)
rfm_df['R_Score'] = pd.qcut(rfm_df['Recency'], q=5, labels=r_labels)

# 3. Hitung F Score (Frequency) - PAKAI TRIK RANKING
rfm_df['F_Score'] = pd.qcut(rfm_df['Frequency'].rank(method='first'), q=5, labels=f_labels)

# 4. Hitung M Score (Monetary)
rfm_df['M_Score'] = pd.qcut(rfm_df['Monetary'], q=5, labels=m_labels)

# 5. Gabungkan Score Jadi Satu String (Contoh: "555")
rfm_df['RFM_Segment'] = rfm_df['R_Score'].astype(str) + rfm_df['F_Score'].astype(str) + rfm_df['M_Score'].astype(str)

# 6. Buat Kolom Total Score (Angka)
rfm_df['RFM_Score_Total'] = rfm_df[['R_Score', 'F_Score', 'M_Score']].sum(axis=1)

# --- CEK HASILNYA ---
print("--- Contoh Hasil Scoring ---")
print(rfm_df[['customer_id', 'Recency', 'Frequency', 'Monetary', 'RFM_Segment', 'RFM_Score_Total']].head())

--- Contoh Hasil Scoring ---
                        customer_id  Recency  Frequency  Monetary RFM_Segment  \
0  00053a61a98854899e70ed204dd4bafe      183          2    382.00         355   
1  0006fdc98a402fceb4eb0ee528f6a8d4      408          1     13.90         111   
2  00082cbe03e478190aadbea78542e933      283          1     79.00         213   
3  00090324bbad0e9342388303bb71ba0a      158          1     49.95         412   
4  000d460961d6dbfa3ec6c9f5805769e1      234          1     28.90         311   

   RFM_Score_Total  
0               13  
1                3  
2                6  
3                7  
4                5  


In [ ]:
# --- FASE 4: PEMBERIAN LABEL MANUSIA ---

def assign_label(score):
    if score >= 13:
        return 'Champions'
    elif score >= 10:
        return 'Loyal'
    elif score >= 7:
        return 'Potential'
    elif score >= 4:
        return 'At Risk'
    else:
        return 'Lost'


rfm_df['Customer_Segment'] = rfm_df['RFM_Score_Total'].apply(assign_label)

# --- CEK HASIL AKHIR ---
print("--- Distribusi Segmen Pelanggan ---")
print(rfm_df['Customer_Segment'].value_counts())

# --- FASE 5: EXPORT KE CSV (UNTUK POWER BI) ---
rfm_df.to_csv('rfm_analysis_result.csv', index=False)

print("\n✅ File 'rfm_analysis_result.csv' berhasil dibuat!")
print("Silakan download file ini dari panel Files di sebelah kiri.")

--- Distribusi Segmen Pelanggan ---
Customer_Segment
Potential    15461
Loyal        12721
At Risk       6220
Champions     3416
Lost           360
Name: count, dtype: int64

✅ File 'rfm_analysis_result.csv' berhasil dibuat!
Silakan download file ini dari panel Files di sebelah kiri.
